## Meteo Bakery: Weather - Feature Engineering

### import packages and configure settings

In [ ]:
# Main data packages. 
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': 'white', 'grid.color': '.8'}
)
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'
cmaps_hex = ['#193251','#FF5A36','#696969', '#7589A2','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

### load weather data

In [ ]:
df = pd.read_csv('../data/neueFische_Wetter.csv')

In [ ]:
df.info()

### Feature Engineering

#### extract x and y components from wind direction

In [ ]:
# Turn wind data into sensible format
df["wind_dir_x"] = df.wind_deg.apply(lambda x: np.cos(np.array(x) * np.pi /180))
df["wind_dir_y"] = df.wind_deg.apply(lambda x: np.sin(np.array(x) * np.pi /180))

In [ ]:
df.wind_gust.fillna(df.wind_speed,inplace=True)
df.rain_1h.fillna(0,inplace=True)
df.snow_1h.fillna(0,inplace=True)

#### Input Weather Codes

In [ ]:
codes = pd.read_csv('../data/neueFische_Wettercodes.csv')
codes.columns = ['weather_id', 'WeatherMain', 'WeatherDescription']

In [ ]:
df = pd.merge(df, codes,how='left',on='weather_id')

#### Extract year, month, day, hour

In [ ]:
df.time = df.apply(lambda x: pd.to_datetime(x['time'],utc=True),axis=1)
df = df.assign(
    #timestamp = lambda x: pd.to_datetime(x['time'],utc=True), 
    date = lambda x: x['time'].dt.date,
    year = lambda x: x['time'].dt.year,
    month = lambda x: x['time'].dt.month,
    week = lambda x: x['time'].dt.isocalendar().week,
    day = lambda x: x['time'].dt.day,
    weekday = lambda x: x['time'].dt.weekday,
    dayofyear = lambda x: x['time'].dt.dayofyear,
    hour = lambda x: x['time'].dt.hour,
)
df.weekday.replace({0:'Mo',
                    1:'Tu',
                    2:'We',
                    3:'Th',
                    4:'Fr',
                    5:'Sa',
                    6:'Su'},inplace=True)

In [ ]:
df.head()

#### Extract daily summary statistics

In [ ]:
# utility function for extracting summary statistics from weather recordings
def extract_daily_statistics(df,columns):
    """Extracts daily summary statistics of hourly weather data. 
        Returns a dataframe with mean, min, max and std values for weather recordings between 06:00 - 20:00 
        and also returns mean values for three daily time period, i.e. 06:00-10:00, 11:00-15:00, and 16:00-20:00

    Args:
        df (pd.DataFrame): Dataframe containing ourly weather recordings
        columns (list): A list of columns containing weather variable names contained in the input dataframe.

    Returns:
        pd.DataFrame: Dataframe containing daily summary statistics of weather data
    """

    # intialize dataframe for summary statistics
    summary_stats = pd.DataFrame({'date': df.date.unique()})
    summary_stats.set_index('date', inplace=True)

    # subselect dataframe for calculating whole-day summary statistics
    df_day = df[df.hour.between(6, 20)]

    # subselect dataframe for specified time frames
    # 06:00-10:00
    df_6_10 = df[df.hour.between(6, 10)]
    # 11:00-15:00
    df_11_15= df[df.hour.between(11, 15)]
    # 16:00-20:00
    df_16_20= df[df.hour.between(16, 20)]

    for col in columns:
        # calculate different summary statistics over complete daytime period
        summary_stats[[col+'_mean', col+'_min', col+'_max', col+'_std']] = df_day.groupby('date').temp.agg(['mean', 'min', 'max', 'std'])
        
        # calculate mean values for specified time frames
        # 06:00-10:00
        df_6_10 = df[df.hour.between(6, 10)]
        summary_stats[col+'_06-10'] = df_6_10.groupby('date').temp.agg(['mean'])
        # 11:00-15:00
        df_11_15= df[df.hour.between(11, 15)]
        summary_stats[col+'_11_15'] = df_11_15.groupby('date').temp.agg(['mean'])
        # 16:00-20:00
        df_16_20= df[df.hour.between(16, 20)]
        summary_stats[col+'_16_20'] = df_16_20.groupby('date').temp.agg(['mean'])
    

    return summary_stats

In [ ]:
df.columns

In [ ]:
summary_stats = extract_daily_statistics(df, ['temp', 'feels_like', 'pressure', 'humidity', 'clouds',
       'visibility', 'wind_speed', 'wind_gust', 'rain_1h','snow_1h', 'wind_dir_x', 'wind_dir_y'])
summary_stats.head()

In [ ]:
summary_stats.info()

In [ ]:
df.date.nunique()

There are as many rows as unqiue days in the summary-statistics df. Thus, the feature engineering seems to be successful.

### Feature engineering on string features

In [ ]:
df.WeatherMain.unique()

In [ ]:
df.WeatherMain.value_counts()

- convert weather main into broader categories (e.g. rain, drizzle == rainy)
- extract mode category as general weather condition for that day

- __broader weather categories__
    * 'Clouds': 'cloudy'
    * 'Snow': 'snowy'
    * 'Rain', 'Drizzle': 'rainy'
    * 'Mist', 'Fog', 'Haze': 'foggy'
    * 'Clear': 'clear'
    * 'Dust', 'Smoke': 'dusty'
    * 'Thunderstorm', 'Tornado': 'stormy'

In [ ]:
# define dictionary to re-classify weather categories
dict = {'Clouds': 'cloudy', 'Snow': 'snowy', 'Rain': 'rainy', 'Mist': 'foggy', 'Drizzle':'rainy', 'Fog':'foggy', 'Clear':'clear',
       'Dust': 'dusty', 'Thunderstorm': 'stormy', 'Haze': 'foggy', 'Smoke': 'dusty', 'Tornado': 'stormy'}
 
 
# Remap the values of the dataframe
df['weather_cats'] = df.WeatherMain.map(dict)

# count frequency per category
df.weather_cats.value_counts()

In [ ]:
summary_stats['condition'] = df.groupby('date').weather_cats.agg(pd.Series.mode)

In [ ]:
summary_stats.head()

In [ ]:
df.WeatherDescription.unique()

In [ ]:
df.WeatherDescription.value_counts()

In [ ]:
summary_stats.to_csv('../data/summary_stats.csv')